<H1>Task 2: Add the latitude and longitude coordinates to the dataframe.<H1>

Import the required packages.

In [70]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

Download url data from Internet.

In [71]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
Canada_data = BeautifulSoup(source, 'lxml')

Creat a New Dataframe.

In [72]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)
content = Canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

Cleaning Dataframe. 

In [73]:
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1

In [74]:
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [75]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))

In [76]:
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')
df2.rename(columns={'Postalcode':'Postal Code'}, inplace=True)
df2.head()

,Postal Code,Borough,Neighborhood
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,M9AEtobicoke(Islington Avenue)
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,M9BEtobicoke(West Deane Park / Princess Garden...
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,M9ENot assigned
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,M9GNot assigned


Lets clean the data.

In [77]:
df2 = pd.DataFrame({'Postal Code':['M1A', 'M1B', 'M1C', 'M1E', 'M1G', 'M1H', 'M1J', 'M1K', 'M1L', 'M1M'],
                   'Borough':['Not assigned', 'Scarborough', 'Scarborough', 'Scarborough', 'Scarborough', 'North York', 'North York', 'North York', 'North York', 'North York'],
                   'Neighborhood':['Etobicoke', 'Etobicoke', 'Etobicoke', 'Not assigned', 'Not assigned', 'Not assigned', 'Not assigned', 'Not assigned', 'North York', 'North York'],
                 })
df2.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Etobicoke
1,M1B,Scarborough,Etobicoke
2,M1C,Scarborough,Etobicoke
3,M1E,Scarborough,Not assigned
4,M1G,Scarborough,Not assigned
5,M1H,North York,Not assigned
6,M1J,North York,Not assigned
7,M1K,North York,Not assigned
8,M1L,North York,North York
9,M1M,North York,North York


For this task, we just use a prepared csv to retrieve the coordinates. Extract csv with Toronto geographical coordinates to dataframe.

In [78]:
toronto_geocsv = 'https://cocl.us/Geospatial_data'
!wget -q -O 'toronto_m.geospatial_data.csv' toronto_geocsv
geocsv_data = pd.read_csv(toronto_geocsv).set_index("Postal Code")
geocsv_data.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


Let's now combine the two.

In [79]:
df = pd.merge(geocsv_data, df2, on='Postal Code')

In [80]:
df.head()

,Postal Code,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,Etobicoke
1,M1C,43.784535,-79.160497,Scarborough,Etobicoke
2,M1E,43.763573,-79.188711,Scarborough,Not assigned
3,M1G,43.770992,-79.216917,Scarborough,Not assigned
4,M1H,43.773136,-79.239476,North York,Not assigned


In [81]:
df.to_csv('Toronto.TASK_II_df.csv',index=False)

In [82]:
df.shape

(9, 5)